**ENGLISH - MARATHI NLP PROJECT**

1.Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

2.Mounting Google Drive

In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


3. Importing The Data

In [ ]:
lines= pd.read_table('/content/drive/MyDrive/NLP pro/eng-mara.txt', names=['eng', 'mar'])

In [ ]:
lines.shape

(41028, 2)

4. Cleaning The Data

In [ ]:
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.mar=lines.mar.apply(lambda x: x.lower())

In [ ]:
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.mar=lines.mar.apply(lambda x: re.sub("'", '', x))

In [ ]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.mar=lines.mar.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [ ]:
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.mar = lines.mar.apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

In [ ]:
# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.mar=lines.mar.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.mar=lines.mar.apply(lambda x: re.sub(" +", " ", x))

In [ ]:
# Add start and end tokens to target sequences
lines.mar = lines.mar.apply(lambda x : 'START_ '+ x + ' _END')

In [ ]:
lines.sample(10)

,eng,mar
11899,it wasnt expensive,START_ महाग नव्हतं _END
8936,are you going too,START_ तूसुद्धा जोतोयस _END
20937,why dont you ever help,START_ तू कधी मदत का नाही करत _END
30417,were leaving in five minutes,START_ आम्ही पाच मिनिटांत निघत आहोत _END
26168,whats your favorite class,START_ तुझा आवडता वर्ग कोणता आहे _END
7471,i know that story,START_ मला ती गोष्ट माहीत आहे _END
20416,this is my first cruise,START_ ही माझी पहिलीच क्रूझ आहे _END
40518,the president of the us paid a formal visit to...,START_ यूएसच्या राष्ट्राध्यक्षाने चीनला औपचारि...
12284,the train just left,START_ ट्रेन नुक्तीच निघाली _END
735,tom failed,START_ टॉम अपयशी ठरला _END


In [ ]:
# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of Marathi
all_marathi_words=set()
for mar in lines.mar:
    for word in mar.split():
        if word not in all_marathi_words:
            all_marathi_words.add(word)

In [ ]:
# Max Length of source sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))
max_length_src = np.max(lenght_list)
max_length_src

34

In [ ]:
# Max Length of target sequence
lenght_list=[]
for l in lines.mar:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

37

In [ ]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_marathi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_marathi_words)
num_encoder_tokens, num_decoder_tokens

(5690, 13722)

In [ ]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

13723

In [ ]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [ ]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [ ]:
lines = shuffle(lines)
lines.head(10)

,eng,mar
35552,tom poured himself a glass of water,START_ टॉमने स्वतःसाठी एक ग्लास पाणी ओतून घेतल...
31509,what did you do with that book,START_ काय केलंत त्या पुस्तकाचं _END
33987,tom didnt have to wait that long,START_ टॉमला तितक्या वेळ थांबावं लागलं नाही _END
633,is this it,START_ एवढच का _END
13246,can you meet tonight,START_ तू मला आज रात्री भेटू शकतेस का _END
39139,will someone tell me what has happened here,START_ कोणी मला सांगेल का की इथे काय झालं आहे ...
14754,tom thinks im crazy,START_ टॉमला वाटतं मी वेडी आहे _END
5066,tom took my car,START_ टॉमने माझी गाडी घेतली _END
5106,watch your feet,START_ पाय सांभाळा _END
39966,i never thought tom would do something like that,START_ टॉम तसलं काहीतरी करू शकेल याचा मी विचार...


4. Splitting the Data For Model

In [ ]:
# Train - Test Split
X, y = lines.eng, lines.mar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((36925,), (4103,))

5. One Hot Encodeing The Data

In [ ]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [ ]:
latent_dim = 50

6. Encoder-Decoder Model Execution

In [ ]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens + 1, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [ ]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 50)     284550      ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 50)     686150      ['input_2[0][0]']                
                                                                                              

In [ ]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 50

In [ ]:
model.fit(generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Epoch 1/50
288/288 [==============================] - 76s 223ms/step - loss: 0.9812 - acc: 0.1774 - val_loss: 0.8838 - val_acc: 0.1974
Epoch 2/50
288/288 [==============================] - 62s 214ms/step - loss: 0.8500 - acc: 0.2091 - val_loss: 0.8328 - val_acc: 0.2288
Epoch 3/50
288/288 [==============================] - 62s 214ms/step - loss: 0.8045 - acc: 0.2423 - val_loss: 0.7995 - val_acc: 0.2532
Epoch 4/50
288/288 [==============================] - 62s 214ms/step - loss: 0.7725 - acc: 0.2609 - val_loss: 0.7740 - val_acc: 0.2692
Epoch 5/50
288/288 [==============================] - 61s 213ms/step - loss: 0.7442 - acc: 0.2802 - val_loss: 0.7485 - val_acc: 0.2870
Epoch 6/50
288/288 [==============================] - 61s 214ms/step - loss: 0.7185 - acc: 0.2993 - val_loss: 0.7272 - val_acc: 0.3052
Epoch 7/50
288/288 [==============================] - 61s 213ms/step - loss: 0.6947 - acc: 0.3174 - val_loss: 0.7078 - val_acc: 0.3213
Epoch 8/50
288/288 [==============================] - 6

In [ ]:
model.save_weights('nmt_weights.h5')

In [ ]:
model.load_weights('nmt_weights.h5')

Encoding & Decoding

In [ ]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

Train Data Predictions

In [ ]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 15ms/step
Input English sentence: she lives in yokohama
Actual Marathi Translation:  ती योकोहामामध्ये राहते 
Predicted Marathi Translation:  ती शहरात राहतात 


Validation Data Prediction

In [ ]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [ ]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

1/1 [==============================] - 0s 20ms/step
Input English sentence: whose books are these
Actual Marathi Translation:  ही पुस्तकं कोणाची आहेत 
Predicted Marathi Translation:  ही पुस्तकं कोणाची आहेत 
